# Imports

In [8]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report

# Nested K-Fold Validation Outline

In [9]:

# Load dataset
data = load_iris()
X, y = data.data, data.target

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

# Initialize the model
model = SVC()

# Outer K-fold cross-validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform nested cross-validation
outer_results = []
for train_idx, test_idx in outer_cv.split(X, y):
    # Split data
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inner loop: Hyperparameter tuning using GridSearchCV
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=outer_cv)
    grid_search.fit(X_train, y_train)

    # Evaluate on the test set
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Collect results
    outer_results.append({
        "best_params": grid_search.best_params_,
        "classification_report": classification_report(y_test, y_pred, output_dict=True),
        "accuracy": best_model.score(X_test, y_test)
    })

# Print the results for each fold
for i, result in enumerate(outer_results, 1):
    print(f"Fold {i}")
    print(f"Best Parameters: {result['best_params']}")
    print("Classification Report:")
    print(classification_report(
        y_test, y_pred, target_names=data.target_names
    ))
    print(f"Accuracy: {result['accuracy']:.4f}")
    print("-" * 40)

# Overall results
mean_accuracy = sum(res["accuracy"] for res in outer_results) / len(outer_results)
print(f"Mean Accuracy: {mean_accuracy:.4f}")


Fold 1
Best Parameters: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.92      1.00      0.96        11
   virginica       1.00      0.92      0.96        12

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

Accuracy: 1.0000
----------------------------------------
Fold 2
Best Parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         7
  versicolor       0.92      1.00      0.96        11
   virginica       1.00      0.92      0.96        12

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

Accur

# Re-Train Single Final Model on entire dataset using best-performing hyperparameters